# VeriThoughts

## SetUp

### Hardware setup


In [1]:
!nvidia-smi

Mon Dec  1 02:55:47 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             46W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
%pip install -U "torch" --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121


### others

In [3]:
! git clone https://github.com/wilyub/VeriThoughts.git

fatal: destination path 'VeriThoughts' already exists and is not an empty directory.


In [4]:
%cd VeriThoughts

/content/VeriThoughts


In [5]:
!sudo apt-get update -qq
!sudo apt-get install -y yosys

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
yosys is already the newest version (0.9-2).
0 upgraded, 0 newly installed, 0 to remove and 57 not upgraded.


In [6]:
!yosys -V

Yosys 0.9 (git sha1 1979e0b)


In [7]:
# ! python multi-turn-yosys-generation.py --start_idx 0 --end_idx 25090 --label_start 0 --batch_file_path your/file/path --gemini_key your_gemini_key --deepseek_key your_deepseek_key --yosys_location your/yosys/location

In [8]:
!ls

evaluation_verithoughts  README.md  verilog-eval
images			 training   verithoughts


In [9]:
%cd verithoughts
! pip install -r requirements.txt

/content/VeriThoughts/verithoughts


In [10]:
!pip install -q -U google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.2/261.2 kB 9.4 MB/s eta 0:00:00


In [11]:
# Step 1: Ask the user for keys securely
import getpass

print("🔐 Please enter your API keys below:")

GEMINI_KEY = getpass.getpass("Enter your Gemini API key: ")
DEEPSEEK_KEY = getpass.getpass("Enter your DeepSeek API key: ")

# Step 2: Confirm they're loaded (don't print actual keys!)
print("✅ Keys stored in memory. Ready to use.")

# Step 3: Export them as environment variables (optional but convenient)
import os
os.environ["GEMINI_KEY"] = GEMINI_KEY
os.environ["DEEPSEEK_KEY"] = DEEPSEEK_KEY


🔐 Please enter your API keys below:
Enter your Gemini API key: ··········
Enter your DeepSeek API key: ··········
✅ Keys stored in memory. Ready to use.


In [13]:
  !python /content/VeriThoughts/verithoughts/multi-turn-yosys-generation.py \
    --start_idx 0 --end_idx 25090 \
    --label_start 0 \
    --batch_file_path /content/VeriThoughts/verithoughts_batches \
    --gemini_key $GEMINI_KEY \
    --deepseek_key $DEEPSEEK_KEY \
    --yosys_location /usr/bin/yosys


Traceback (most recent call last):
  File "/content/VeriThoughts/verithoughts/multi-turn-yosys-generation.py", line 273, in <module>
    question_list = [question_prompt + rtl['ground_truth'] for rtl in rtl_list if len(rtl['ground_truth']) < 10000]
                                                                                      ~~~^^^^^^^^^^^^^^^^
TypeError: string indices must be integers, not 'str'


Since the new generated datasets (list) don't match the expected data type (dict) for yosys.


Hence, I'll instead use the given Huggingface dataset.

## Huggingface Dataset

In [14]:
!pip -q install "datasets>=2.18" "transformers>=4.43" "accelerate" "bitsandbytes" "huggingface_hub"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 26.0 MB/s eta 0:00:00


In [15]:
from datasets import load_dataset

# Full training set used by the paper (contains reasoning and the verification label)
ds = load_dataset("wilyub/VeriThoughtsTrainSet", split="train")

print(ds)
print(ds[0].keys())  # expect: ground_truth, question, generated_verilog, reasoning_trace, verified, ...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/20.0 [00:00<?, ?B/s]

hf_complete_trainset.jsonl:   0%|          | 0.00/399M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20173 [00:00<?, ? examples/s]

Dataset({
    features: ['ground_truth', 'question', 'generated_verilog', 'verified', 'reasoning_trace'],
    num_rows: 20173
})
dict_keys(['ground_truth', 'question', 'generated_verilog', 'verified', 'reasoning_trace'])


In [16]:
consistent   = ds.filter(lambda r: r["verified"] is True)
inconsistent = ds.filter(lambda r: r["verified"] is False)

len(consistent), len(inconsistent)

Filter:   0%|          | 0/20173 [00:00<?, ? examples/s]

Filter:   0%|          | 0/20173 [00:00<?, ? examples/s]

(10069, 10104)

For now, we won't fine-tune our own model, but rather call pre-trained models, and evaluate them.

## Evaluation

### 0) One-time setup

In [17]:
%cd /content/VeriThoughts/evaluation_verithoughts

/content/VeriThoughts/evaluation_verithoughts


In [19]:
# !pip -q install -r requirements.txt

### 1) Download the benchmark set from Hugging Face

In [20]:
from huggingface_hub import snapshot_download
bench_dir = "./verithoughts_benchmark"

snapshot_download(repo_id="wilyub/VeriThoughtsBenchmark",
                  local_dir=bench_dir,
                  repo_type="dataset",
                  tqdm_class=None)

print("Benchmark at:", bench_dir)


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/20.0 [00:00<?, ?B/s]

hf_benchmark.jsonl: 0.00B [00:00, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

Benchmark at: ./verithoughts_benchmark


### 2) Choose a pretrained model

Good starting points:

nyu-dice-lab/VeriThoughts-Reasoning-7B (fits on Colab with 8-bit)

nyu-dice-lab/VeriThoughts-Reasoning-14B (needs more VRAM)

nyu-dice-lab/VeriThoughts-Instruct-7B

### 3) Generate benchmark responses (as the README shows)
This writes outputs.jsonl in the current folder.

In [21]:
# 1) See what file(s) were downloaded into the benchmark folder
import glob, os, textwrap

root = "./verithoughts_benchmark"
files = glob.glob(os.path.join(root, "**", "*.json*"), recursive=True)
print("Found JSON files:\n", "\n".join(files))


Found JSON files:
 ./verithoughts_benchmark/hf_benchmark.jsonl


In [26]:
# Create a smaller benchmark with just 24 questions
!head -3 ./verithoughts_benchmark/hf_benchmark.jsonl > ./small_benchmark.jsonl

!python verilog_vllm_multi.py \
  --benchmark_path ./small_benchmark.jsonl \
  --model_id nyu-dice-lab/VeriThoughts-Reasoning-7B \
  --sample_number 1 \
  --batch_size 8 \
  --reasoning_mode \
  --tensor_parallel_size 1 \
  --output_path ./outputs.jsonl
#   --benchmark_path ./verithoughts_benchmark/hf_benchmark.jsonl \



2025-12-01 04:42:33.115230: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-01 04:42:33.132814: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764564153.154224   33211 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764564153.160700   33211 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764564153.177011   33211 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

### 4) Evaluate formally with Yosys

In [29]:
!python evaluation_script.py \
  --results_path ./benchmark_results/nyu-dice-lab/VeriThoughts-Reasoning-7B/results.jsonl \
  --samples_per_question 1 \
  --yosys_location /usr/bin/yosys \
  --benchmark_path ./small_benchmark.jsonl

pass@1: 0.0
pass@5: 0.0
pass@10: 0.0
